# Citation Dataset Loading

In this notebook we're going to load the citation dataset into Neo4j.

First let's import a couple of Python libraries that will help us with this process.

In [1]:
!pip install py2neo pandas

We'll start by importing py2neo library which we'll use to import the data into Neo4j. py2neo is a client library and toolkit for working with Neo4j from within Python applications. It is well suited for Data Science workflows and has great integration with other Python Data Science tools.

In [1]:
from py2neo import Graph

Create a [Neo4j 3.4 Sandbox](https://neo4j.com/sandbox-v2/) and paste the credentials into the cell below.


<div align="left">
    <img src="https://github.com/mneedham/data-science-training/blob/master/images/sandbox.png?raw=1" alt="Neo4j 3.4 Sandbox"/>
</div>


In [2]:
# Change the line of code below to use the IP Address, Bolt Port, and Password of your Sandbox.
# graph = Graph("bolt://<IP Address>:<Bolt Port>", auth=("neo4j", "<Password>")) 

# graph = Graph("bolt://18.234.168.45:33679", auth=("neo4j", "daybreak-cosal-rumbles")) 
graph = Graph("bolt://localhost:7687", auth=("neo4j", "12345678")) 

## Create Constraints

First let's create some constraints to make sure we don't import duplicate data:

In [3]:
display(graph.run("CREATE CONSTRAINT ON (a:Article) ASSERT a.index IS UNIQUE").stats())
display(graph.run("CREATE CONSTRAINT ON (a:Author) ASSERT a.name IS UNIQUE").stats())
display(graph.run("CREATE CONSTRAINT ON (v:Venue) ASSERT v.name IS UNIQUE").stats())

{'constraints_added': 1}

{'constraints_added': 1}

{'constraints_added': 1}

## Loading the data

Now let's load the data into the database. We'll create nodes for Articles, Venues, and Authors.


In [4]:
query = """
CALL apoc.periodic.iterate(
  'UNWIND ["dblp-ref-0.json", "dblp-ref-1.json", "dblp-ref-2.json", "dblp-ref-3.json"] AS file
   CALL apoc.load.json("https://github.com/mneedham/link-prediction/raw/master/data/" + file)
   YIELD value WITH value
   return value',
  'MERGE (a:Article {index:value.id})
   SET a += apoc.map.clean(value,["id","authors","references", "venue"],[0])
   WITH a, value.authors as authors, value.references AS citations, value.venue AS venue
   MERGE (v:Venue {name: venue})
   MERGE (a)-[:VENUE]->(v)
   FOREACH(author in authors | 
     MERGE (b:Author{name:author})
     MERGE (a)-[:AUTHOR]->(b))
   FOREACH(citation in citations | 
     MERGE (cited:Article {index:citation})
     MERGE (a)-[:CITED]->(cited))', 
   {batchSize: 1000, iterateList: true});
"""
graph.run(query).to_data_frame()

,batches,total,timeTaken,committedOperations,failedOperations,failedBatches,retries,errorMessages,batch,operations,wasTerminated,failedParams
0,52,51956,153,51956,0,0,0,{},"{'total': 52, 'committed': 52, 'failed': 0, 'e...","{'total': 51956, 'committed': 51956, 'failed':...",False,{}


In the next notebook we'll explore the data that we've imported. 